In [9]:
import numpy as np 
import pandas as pd 
import pandas as pd
import numpy as np
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_scheduler
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [12]:
# BEGIN: Load datasets
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')
# END: Load datasets
# BEGIN: Analyze misconceptions
# Preprocessing function
def preprocess_data(df):
    # Fill missing values in StudentExplaination and Misconception with empty strings
    df['StudentExplaination'] = df['StudentExplanation'].fillna("")
    df['Misconception'] = df['Misconception'].fillna("NA")
    
    # Combine QuestionText and StudentExplaination for feature extraction
    df['CombinedText'] = df['QuestionText'] + " " + df['StudentExplaination']
    
    return df

# Preprocess train and test data
train_df = preprocess_data(train_df)
#test_df = preprocess_data(test_df)

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train = vectorizer.fit_transform(train_df['CombinedText']).toarray()
#X_test = vectorizer.transform(test_df['CombinedText']).toarray()

# Encode the Category column as the label
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['Category'])
#y_test = label_encoder.transform(test_df['Category'])

# Misconception column for generating misconceptions
misconceptions_train = train_df['Misconception']

#misconceptions_test = test_df['Misconception']

# Split train data into training and validation sets
X_train, X_val, y_train, y_val, misconceptions_train, misconceptions_val = train_test_split(
    X_train, y_train, misconceptions_train, test_size=0.2, random_state=42
)

# Output the shapes of the datasets
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
#print("Test data shape:", X_test.shape)
print("Labels shape:", y_train.shape)

Training data shape: (29356, 5000)
Validation data shape: (7340, 5000)
Labels shape: (29356,)
